# LGBIO2110 - Introduction to clinical engineering
## Project : Epidemiology modelling of COVID-19

__Date :__ Year 2021-2022

__Professor :__ Philippe Lefèvre and Benoît Delhaye

__Author :__ Benoît Delhaye and Donatien Doumont

__Content :__ At the end of this project, you should master and understand the following :


*   Master and be able to use mathematical modelling of a epidemy
*   Understanding the mechanisms that influence the disease spread and their dynamics. Be able to relate the dynamical evolution of the epidemy to events (political decisions, appearance of virus variant, vaccination,...) 
*   Predict the future trend : how severe will the pandemic be ? 
*   Suggest control strategies and evaluate their effects


In [43]:
#Librairies utiles 
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import numpy as np
import urllib.request
import pandas as pd
import ipywidgets as widgets
%config InlineBackend.figure_format = 'retina'

# use NMA plot style
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")
my_layout = widgets.Layout()

<font size=5 color=#009999> Context  </font> <br> <br>
SIR model from Kermack-McKendrick is expressed using only three *communicating vessel*
*   Susceptible S(t)
*   Infected I(t)
*   Removed R(t) caused by isolation/recovery/death

Dynamical equations of the model are the following : 
\begin{align*}
    \dot{S} &= -\beta S I\\
    \dot{I} &= \beta S I - \alpha I\\
    \dot{R} &= \alpha I\\
\end{align*}
Remark that the hypothesis of a constant population is made here, hence $\dot{S} + \dot{I} + \dot{R} = 0$ is verified.

***
Example : Behavior of the model for a given initial value for the susceptible $S(t)$ and infected $I(t)$ group, and for constant beta $\beta$ and alpha $\alpha$ parameters.
***
Play with the widgets parameters to better interpret their impact on the model.

In [44]:
#SIR model discrete computation 
def SIRmodel(S0,I0,param):
    beta = param[0]
    alpha = param[1]
    dt = param[2]
    #Initialisation
    S = np.zeros((N,1))
    I = S.copy()
    R = S.copy() 
    S[0] = S0 
    I[0] = I0
    for i in range(0,N-1) : 
        #print(i)
        S[i+1] = S[i]+(-beta*S[i]*I[i])*dt
        I[i+1] = I[i]+(beta*S[i]*I[i]-alpha*I[i])*dt
        R[i+1] = R[i]+(alpha*I[i])*dt
    return S, I, R

#Graphical representation
def plot_SIRmodel(t_vector, S, I, R):
    plt.plot(t_vector, S, label = 'S(t)')
    plt.plot(t_vector, I, label = 'I(t)')
    plt.plot(t_vector, R, label = 'R(t)')
    plt.legend()
    plt.xlabel("Time [days]")
    plt.ylabel("Relative group size")
    plt.xlim((0,t_vector[-1]))
    plt.ylim((0,1))
    

In [45]:
#--------------------------------------------------
tend = 60 #in days
Fs = 1e2 #sample frequency
N = int(tend*Fs) #sample size
time = np.linspace(0,tend,N) #time sequence

#Initial condition of the epidemic
S0 = 0.9
I0 = 0.1

def refresh(beta=0.5,alpha_over_one=10):
    #Dynamical evolution of the model
    alpha = 1/alpha_over_one
    Repro0 = beta*S0/alpha
    S,I,R = SIRmodel(S0,I0,[beta, alpha, 1/Fs])
    plot_SIRmodel(time, S, I, R)
    plt.title("$R_0$ = " + str(Repro0))
    plt.show()
style = {'description_width' : 'initial'}

_ = widgets.interact(refresh,
    beta = widgets.FloatLogSlider(value=0.5, min=-2, max=1, steps=0.05, description="Logarithmic slider: beta", style = style),
    alpha_over_one = widgets.IntSlider(value=10, min=1, max=30, step=1, description="Linear slider: 1/alpha(in days):",style=style),
)

interactive(children=(FloatLogSlider(value=0.5, description='Logarithmic slider: beta', max=1.0, min=-2.0, sty…

<font size=5 color=#009999> Homework  </font> <br> <br>
Instructions : 
1. Collect Belgian data 
2. Adjust the infected population number (being underestimated in the collected dataset)
3. Estimate the time varying reproduction number $R_t$ with an appropiate simple model and show the effect of government measures on the time variyng $R_t$ 
4. Forecast the evolution of $\beta(t)$ and predict the evolution of the pandemic 
5. Include the effect of vaccine 


The dataset used in this project is restricted to the following time period : from $t_0$ = **March, 2020** to $t_1$ = **March, 2021**. 

You will have to predict the behaviour of the epidemy at least **three months** after $t_1$. 

***
1) Importation of dataset
***
Open accessed data are imported from webpage sciensano and are preprocessed. 

Rem : The dataset is created in csv file in a folder data

In [46]:
def create_df(l):
    """
    :param l: name of interested data to complete path to the file
    :return: Two dataframes. One with no modification and the other one where data are grouped 
    according to the date
    """
    fnloc = "dataset/COVID19BE_" + l + ".csv"
    df = pd.read_csv(fnloc, sep=",", header='infer', parse_dates=["DATE"])
    if l == "VACC":
        df.loc[:, "DOSEA"] = (df.DOSE == "A") * df.COUNT
        df.loc[:, "DOSEB"] = (df.DOSE == "B") * df.COUNT
        df.loc[:, "DOSEC"] = (df.DOSE == "C") * df.COUNT
        df.loc[:, "DOSEE"] = (df.DOSE == "E") * df.COUNT

    # Keep only numerical quantities in the dataset
    fieldn = df.columns
    dfnum = df.select_dtypes(include='number')
    vars = dfnum.columns
    dfnum.insert(0, 'DATE', df.DATE, True)

    # Sum values over the same day
    dfsum = dfnum.groupby(['DATE'], as_index=True).sum()
    for ii in range(len(vars)):
        exec(l + '_' + vars[ii] + ' = dfsum.' + vars[ii])
    exec(l + '_DATE = df.DATE.unique()')
    return df, dfsum

In [47]:
fn = ["HOSP", "MORT", "CASES_AGESEX", "tests", "VACC"]

df_hosp,dfsum_hosp = create_df("HOSP")
df_dead,dfsum_dead = create_df("MORT")
df_case,dfsum_case = create_df("CASES_AGESEX")
df_test,dfsum_test = create_df("tests")
df_vacc,dfsum_vacc = create_df("VACC")

Variable available across time :
- from HOSP :           HOSP_DATE, HOSP_NEW_IN, HOSP_NEW_OUT, HOSP_NR_REPORTING, HOSP_TOTAL_IN, HOSP_TOTAL_IN_ECMO, HOSP_TOTAL_IN_ICU, HOSP_TOTAL_IN_RESP.
- from MORT :           MORT_DATE, MORT_DEATHS.
- from CASES_AGESEX :   CASES_AGESEX_DATE, CASES_AGESEX_CASES.
- from tests :          tests_DATE, tests_TESTS_ALL, tests_TESTS_ALL_POS.
- from VACC :           VACC_DATE, VACC_COUNT, VACC_DOSEA, VACC_DOSEB, VACC_DOSEC, VACC_DOSEE.

Beware that each 
***
2. Infected population estimation : The goal is to better estimate the infected population begin underestimated in the dataset.  
***

In [48]:
display(dfsum_hosp)

,NR_REPORTING,TOTAL_IN,TOTAL_IN_ICU,TOTAL_IN_RESP,TOTAL_IN_ECMO,NEW_IN,NEW_OUT
DATE,,,,,,,
2020-03-15,104,263,53,31,1,70,14
2020-03-16,104,370,79,51,1,88,14
2020-03-17,104,493,100,65,3,125,28
2020-03-18,104,646,130,88,4,179,26
2020-03-19,104,840,164,114,4,213,49
...,...,...,...,...,...,...,...
2022-03-13,103,1999,175,78,11,102,103
2022-03-14,103,2103,177,75,10,162,117
2022-03-15,103,2218,181,77,9,200,310


In [49]:
display(dfsum_dead)

,DEATHS
DATE,
2020-03-07,1
2020-03-10,1
2020-03-11,3
2020-03-13,3
2020-03-14,7
...,...
2022-03-13,12
2022-03-14,23
2022-03-15,14


In [50]:
display(dfsum_case)

,CASES
DATE,
2020-03-01,19
2020-03-02,19
2020-03-03,34
2020-03-04,53
2020-03-05,81
...,...
2022-03-13,3987
2022-03-14,15602
2022-03-15,12159


In [51]:
display(dfsum_test)

,TESTS_ALL,TESTS_ALL_POS
DATE,,
2020-03-01,82,0
2020-03-02,317,10
2020-03-03,538,21
2020-03-04,701,37
2020-03-05,773,65
...,...,...
2022-03-13,16027,4017
2022-03-14,42397,11984
2022-03-15,51984,14984


In [52]:
display(dfsum_vacc)

,COUNT,DOSEA,DOSEB,DOSEC,DOSEE
DATE,,,,,
2020-12-28,339,339,0,0,0
2020-12-29,40,29,1,10,0
2020-12-30,529,519,1,9,0
2020-12-31,48,48,0,0,0
2021-01-01,17,17,0,0,0
...,...,...,...,...,...
2022-03-12,9178,457,4264,0,2428
2022-03-13,1511,48,1006,0,188
2022-03-14,1512,77,477,0,525


In [53]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=2, subplot_titles=("Daily cases", "Hospital IN", "Deaths", "Nb of tests")
)

# Add traces
fig.add_trace(go.Scatter(x=dfsum_case.index, y=dfsum_case["CASES"],name="daily case"), row=1, col=1)
fig.add_trace(go.Scatter(x=dfsum_hosp.index, y=dfsum_hosp["TOTAL_IN"],name="Total In Hospital"), row=1, col=2)
fig.add_trace(go.Scatter(x=dfsum_dead.index, y=dfsum_dead["DEATHS"],name="daily death"), row=2, col=1)
fig.add_trace(go.Scatter(x=dfsum_test.index, y=dfsum_test["TESTS_ALL"],name="daily tests"), row=2, col=2)

# Update title and height
fig.update_layout(title_text="Covid19 data for Belgium", height=700)
fig.write_html("figures/brut_data.html")
fig.show()

### Smooth data

Data from scienscano are calculated for each week, so it can be inteeresting to use a Moving Average to smooth data. 

In [54]:
dfsum_case["CASES"] = dfsum_case["CASES"].rolling(7,min_periods=1).mean()
dfsum_dead["DEATHS"] = dfsum_dead["DEATHS"].rolling(7,min_periods=1).mean()
dfsum_test["TESTS_ALL"] = dfsum_test["TESTS_ALL"].rolling(7,min_periods=1).mean()
dfsum_hosp["TOTAL_IN"] = dfsum_hosp["TOTAL_IN"].rolling(7,min_periods=1).mean()

In [55]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=2, cols=2, subplot_titles=("Daily cases", "Hospital IN", "Deaths", "Nb of tests")
)

# Add traces
fig.add_trace(go.Scatter(x=dfsum_case.index, y=dfsum_case["CASES"],name="daily case"), row=1, col=1)
fig.add_trace(go.Scatter(x=dfsum_hosp.index, y=dfsum_hosp["TOTAL_IN"],name="Total In Hospital"), row=1, col=2)
fig.add_trace(go.Scatter(x=dfsum_dead.index, y=dfsum_dead["DEATHS"],name="daily death"), row=2, col=1)
fig.add_trace(go.Scatter(x=dfsum_test.index, y=dfsum_test["TESTS_ALL"],name="daily tests"), row=2, col=2)

# Update title and height
fig.update_layout(title_text="Covid19 data for Belgium", height=700)
fig.write_html("figures/smooth_data.html")
fig.show()

### Adjust the infected population number 

We use the number of new cases in the hospital to adjust the number of infected people:
$$ I(t) = Total\_in\_hospital(t) * k $$ 
We utilize data from a period when the test strategy was better than it was at the onset of the pandemic to discover the best k. The time span between October 20, 2021 and December 31, 2021 appears to be ideal.

In [56]:
t0_k = '10/16/2021'
t1_k = '12/28/2021'
range_T0_T1_k = pd.date_range(start=t0_k, end=t1_k).to_list()
lst_range_k = []
for time in range_T0_T1_k:
    lst_range_k.append(time.strftime("%Y-%m-%d"))

In [57]:
defsum_case_t0_t1_k = dfsum_case.loc[lst_range_k]
defsum_hosp_t0_t1_k = dfsum_hosp.loc[lst_range_k]

In [58]:
defsum_hosp_t0_t1_k_array = np.array(defsum_hosp_t0_t1_k["TOTAL_IN"])
defsum_case_t0_t1_k_array = np.array(defsum_case_t0_t1_k["CASES"])

In [59]:
print("min of k:",np.min(defsum_case_t0_t1_k_array/defsum_hosp_t0_t1_k_array))
print("max of k:",np.max(defsum_case_t0_t1_k_array/defsum_hosp_t0_t1_k_array))
print("mean of k:",np.mean(defsum_case_t0_t1_k_array/defsum_hosp_t0_t1_k_array))

min of k: 2.553768739949162
max of k: 5.777161076995748
mean of k: 4.508756560412957


In [60]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=defsum_case_t0_t1_k.index, y=defsum_case_t0_t1_k["CASES"], name="Cases"))
fig1.add_trace(go.Scatter(x=defsum_hosp_t0_t1_k.index, y=defsum_hosp_t0_t1_k["TOTAL_IN"], name="Hospitalisation k =1"))
fig1.add_trace(go.Scatter(x=defsum_hosp_t0_t1_k.index, y=defsum_hosp_t0_t1_k["TOTAL_IN"]*2.6, name="Hospitalisation k =2.6"))
fig1.add_trace(go.Scatter(x=defsum_hosp_t0_t1_k.index, y=defsum_hosp_t0_t1_k["TOTAL_IN"]*4.5, name="Hospitalisation k =4.5"))
fig1.add_trace(go.Scatter(x=defsum_hosp_t0_t1_k.index, y=defsum_hosp_t0_t1_k["TOTAL_IN"]*5.8, name="Hospitalisation k =5.8"))
fig1.write_html("figures/adjusted_infected.html")
fig1.update_layout(title_text="Covid19: Adjust the infected population number", height=700)
fig1.show()

### Important constants

In [73]:
k = 5.8
N_pop = 11492641  #value from https://en.wikipedia.org/wiki/Belgium
t0 = '3/15/2020'
t1 = '2/28/2021'
range_T0_T1 = pd.date_range(start=t0, end=t1).to_list()
N=len(range_T0_T1) 

lst_range = []
for time in range_T0_T1:
    lst_range.append(time.strftime("%Y-%m-%d"))

In [74]:
dfsum_vacc=dfsum_vacc.reindex(range_T0_T1, fill_value=0)
dfsum_case=dfsum_case.reindex(range_T0_T1, fill_value=0)
dfsum_test=dfsum_test.reindex(range_T0_T1, fill_value=0)
dfsum_hosp=dfsum_hosp.reindex(range_T0_T1, fill_value=0)
dfsum_dead=dfsum_dead.reindex(range_T0_T1, fill_value=0)

In [75]:
defsum_case_t0_t1 = dfsum_case.loc[lst_range]
defsum_hosp_t0_t1 = dfsum_hosp.loc[lst_range]
defsum_vacc_t0_t1 = dfsum_vacc.loc[lst_range]
defsum_dead_t0_t1 = dfsum_dead.loc[lst_range]
defsum_test_t0_t1 = dfsum_test.loc[lst_range]

In [78]:
infected_t0_t1 = np.array(defsum_hosp_t0_t1["TOTAL_IN"] *k)/N_pop
removed_t0_t1 = np.zeros(len(infected_t0_t1))
removed_t0_t1[1:] = np.cumsum(infected_t0_t1)[:-1]
susceptible_t0_t1 = np.ones(len(infected_t0_t1))
susceptible_t0_t1 -= (infected_t0_t1 +removed_t0_t1)

In [79]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=lst_range, y=infected_t0_t1, name="infected"))
fig1.add_trace(go.Scatter(x=lst_range, y=susceptible_t0_t1, name="susceptible"))
fig1.add_trace(go.Scatter(x=lst_range, y=removed_t0_t1, name="removed"))

fig1.update_layout(title_text="Covid19: t0 to t1", height=700)
fig1.write_html("figures/SIR_data.html")
fig1.show()

***
3) Fit Data : The goal here is to fit the beta coefficient $\beta(t)$ with the data across time.
You will need to improve your model from the basic SIR model to a SEIR model that account for exposed population group. This group is infected but not contagious for a period of time. 
***

To calculate the beta : 
$$S[i+1] = S[i]+(-beta*S[i]*I[i])*dt$$
$$-\frac{S[i+1]-S[i]}{S[i]*I[i]*dt}$$

In [80]:
beta = np.zeros(len(infected_t0_t1)-1)
for i in range(0,len(infected_t0_t1)-1):
    beta[i] = -(susceptible_t0_t1[i+1]-susceptible_t0_t1[i])/(susceptible_t0_t1[i]*infected_t0_t1[i])

In [82]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=lst_range[:-1], y=beta, name="beta"))

fig1.update_layout(title_text="Covid19: beta", height=700)
fig1.write_html("figures/SIR_data.html")
fig1.show()

In [83]:
print("mean of beta",np.mean(beta))
print("std of beta",np.std(beta))

mean of beta 1.2443531479335361
std of beta 0.16139412217879257


***
4) Predict the future : The goal is to make prediction about the evolution of beta after $t_1$
*** 

In [ ]:
## Predict the future based on your model

######################
### Your code here ###
######################

***
5) Improve the model to take into account additional features.
***

In [ ]:
## Improvements 

######################
### Your code here ###
######################